In [1]:
# !pip3 install google-cloud-aiplatform --upgrade --user

In [2]:
from google.cloud import aiplatform
from google.cloud.aiplatform import hyperparameter_tuning as hpt

In [3]:
# The spec of the worker pools including machine type and Docker image
# Be sure to replace IMAGE_URI with the path to your Docker image in GCR
worker_pool_specs = [{
                    "machine_spec": {
                                    "machine_type": "n1-standard-4",
                                    "accelerator_type": "NVIDIA_TESLA_T4",
                                    "accelerator_count": 1
                                    },
                    "replica_count": 1,
                    "container_spec": {
                                        "image_uri": "gcr.io/kubeflow-1-0-2/card-fraud-vertex-ai:hypertune"  # <-- Change this
                                      }
                    }]


# Dicionary representing metrics to optimize.
# The dictionary key is the metric_id, which is reported by your training job,
# And the dictionary value is the optimization goal of the metric.
metric_spec={'roc_auc':'maximize'}

# Dictionary representing parameters to optimize.
# The dictionary key is the parameter_id, which is passed into your training
# job as a command line argument,
# And the dictionary value is the parameter specification of the metric.
parameter_spec = {
                "learning_rate": hpt.DoubleParameterSpec(min=0.001, max=0.1,  # float
                                                         scale="log"),
                "max_depth": hpt.DiscreteParameterSpec(values=[5, 10, 15, 20],  # int
                                                       scale=None),
                "scale_pos_weight": hpt.DiscreteParameterSpec(values=[1, 3, 5, 7],  # int
                                                              scale=None)
                }

In [4]:
# Replace YOUR_BUCKET
my_custom_job = aiplatform.CustomJob(display_name='card-fraud-xgboost-hptune',
                                     worker_pool_specs=worker_pool_specs,
                                     staging_bucket='gs://card-fraud-hpt')  # <-- Change this

In [5]:
hp_job = aiplatform.HyperparameterTuningJob(
                                            display_name='card-fraud-xgboost-hptune',
                                            custom_job=my_custom_job,
                                            metric_spec=metric_spec,
                                            parameter_spec=parameter_spec,
                                            max_trial_count=15,
                                            parallel_trial_count=3
                                            )

hp_job.run()

INFO:google.cloud.aiplatform.jobs:Creating HyperparameterTuningJob
INFO:google.cloud.aiplatform.jobs:HyperparameterTuningJob created. Resource name: projects/9118975290/locations/us-central1/hyperparameterTuningJobs/4621269361775083520
INFO:google.cloud.aiplatform.jobs:To use this HyperparameterTuningJob in another session:
INFO:google.cloud.aiplatform.jobs:hpt_job = aiplatform.HyperparameterTuningJob.get('projects/9118975290/locations/us-central1/hyperparameterTuningJobs/4621269361775083520')
INFO:google.cloud.aiplatform.jobs:View HyperparameterTuningJob:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/4621269361775083520?project=9118975290
INFO:google.cloud.aiplatform.jobs:HyperparameterTuningJob projects/9118975290/locations/us-central1/hyperparameterTuningJobs/4621269361775083520 current state:
JobState.JOB_STATE_RUNNING
INFO:google.cloud.aiplatform.jobs:HyperparameterTuningJob projects/9118975290/locations/us-central1/hyperparameterTuningJobs/4621269361